<a href="https://colab.research.google.com/github/shaqas86/Langgraph_EcoSystem/blob/main/RAG_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install -qU pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
userdata.get('GEMINI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [ ]:
# Ensure your VertexAI credentials are configured
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm.invoke("Agent Developer greet me!")

AIMessage(content='Hello there!  How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ea507996-8c1c-4834-a6f1-ad3bb234854b-0', usage_metadata={'input_tokens': 6, 'output_tokens': 12, 'total_tokens': 18, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#create embedding means numeric representation of text provided or document
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
em=embeddings.embed_query("hello")
# em
em[:3]


[0.017157701775431633, -0.009304771199822426, -0.011424741707742214]

In [ ]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
# os.environ['PINECONE_ENV'] = userdata.get('pinecone_env')
api_key = os.environ.get('PINECONE_API_KEY')


In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "rag-dlims" # name must be lower case and ''
pc = Pinecone(api_key=api_key)

pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)
print(index)
vector_store = PineconeVectorStore(embedding=embeddings, index=index) #Vector Store

In [ ]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader(file_path)
# pages = []
# async for page in loader.alazy_load():
#     pages.append(page)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pprint import pprint
# Load documents
loader = TextLoader("/content/dlims.txt")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs = text_splitter.split_documents(documents) # documents splitted
pprint(docs)

[Document(metadata={'source': '/content/dlims.txt'}, page_content='Driving License Issuance Management System (DLIMS) - Punjab, Pakistan\n\nOverview:\nThe Driving License Issuance Management System (DLIMS) is an initiative by the Government of Punjab, Pakistan, aimed at automating the processes of driving license issuance, renewal, and upgrades across the province. This centralized system enhances efficiency, transparency, and security in managing driving licenses.'),
 Document(metadata={'source': '/content/dlims.txt'}, page_content='Key Features:\n- Centralized License Management: DLIMS provides a unified platform for the issuance and management of driving licenses throughout Punjab, ensuring uniformity and reducing discrepancies. [Source: https://dlims.punjab.gov.pk/]\n\n- Automated Processes: The system automates various procedures, including application submission, testing, and license issuance, minimizing manual interventions and errors. [Source: https://dlims.punjab.gov.pk/]'),
 

In [ ]:
len(docs)
# docs[Document(metadata, page_content)]

8

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
# vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content) #create embedding numeric

    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content)) #uuid or Hash doc_id = 22i3y12jdyuew****

    # Create metadata dictionary
    # doc_metadata = {"source": doc.metadata["source"]}
    documents_list=[doc]
    # Upsert the vector with a unique ID and metadata
    # index.upsert(vectors=[(doc_id, vector, doc_metadata)]) # Include metadata in upsert
    vector_store.add_documents(documents=documents_list, id=doc_id, embedding=vector)

100%|██████████| 8/8 [00:03<00:00,  2.40it/s]


In [ ]:
from pprint import pprint

user_query="What is DLIMS"
results = vector_store.similarity_search_with_score(
    query=user_query,
    k=1, #most similar closest vector
    # filter={"source": "/content/dlims.txt"}
    )
if results is not None:
    pprint(results)
else:
    print("No relevant documents found for the query.")
# for res in results:
#     print(f"*{res.page_content},[{res.metadata}]")
# print(results)

[(Document(id='45181ef6-5234-4625-827c-789fadf1149c', metadata={'source': '/content/dlims.txt'}, page_content='Key Features:\n- Centralized License Management: DLIMS provides a unified platform for the issuance and management of driving licenses throughout Punjab, ensuring uniformity and reducing discrepancies. [Source: https://dlims.punjab.gov.pk/]\n\n- Automated Processes: The system automates various procedures, including application submission, testing, and license issuance, minimizing manual interventions and errors. [Source: https://dlims.punjab.gov.pk/]'),
  0.752459586)]


In [ ]:
user_query= user_query
results = results
prompt_template=f"DLIMS RAG context is {results} and user query is {user_query}"
llm.invoke(prompt_template)

In [38]:
from langchain.chains import RetrievalQA
# prompt = f"DLIMS RAG context is {results} and user query is {user_query}"
user_query = "steps to apply for Learner"
# vector_query = embeddings.embed_query(user_query)
results = vector_store.similarity_search(
    user_query,
    k=1,
    # filter={"source": "/content/dlims.txt"}
    )
prompt_template =f"DLIMS RAG context is {results} and user query is {user_query}"

if results is not None:
    response = llm.invoke(prompt_template+"Answer the query")
    # response = results
    pprint(response.content)
else:
    print("No relevant documents found for the query.")


('To apply, follow these steps:\n'
 '\n'
 '1. **Create an account:**  Create an account on the DLIMS online portal at '
 '[https://dlims.punjab.gov.pk/](https://dlims.punjab.gov.pk/).\n'
 '\n'
 '2. **Submit the application form:**  Complete the application form '
 'accurately, providing all required information.\n'
 '\n'
 '3. **Upload documents:** Upload all necessary documents as specified in the '
 'application requirements.\n')
